In [1]:
import pickle
import time
import csv
from selenium import webdriver 
from selenium.webdriver.common.by import By

import pandas as pd 

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
options.add_argument('user-agent=' + user_agent)
options.add_argument('headless')

driver = webdriver.Chrome('C:/final/final_sh/chromedriver.exe', options=options)

C:\Users\admin\AppData\Local\Temp\ipykernel_15996\3268206848.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/final/final_sh/chromedriver.exe', options=options)


In [31]:
df = pd.read_csv('C:/final/final_sh/crawling_re.csv', encoding='utf-8')

In [32]:
keyword_list_sohee=df['travel_busan'].to_list()

In [33]:
keyword_list_sohee

['금정산',
 '낙동강하구',
 '두도',
 '두송반도',
 '몰운대',
 '백양산',
 '송도반도',
 '오륙도',
 '이기대',
 '장산',
 '태종대',
 '가덕도 외양포',
 '감천항',
 '갤러리 미고',
 '광복로 연등문화제',
 '다대포 실내수영장',
 '씨프랜드',
 '일광낭만가요제',
 '파라다이스 카지노 부산',
 '학리방파제']

In [34]:
with open('keyword_list_sh.pkl', 'wb') as f3:
    pickle.dump(keyword_list_sohee, f3) 

In [35]:
# url_crawler
cnt = 0
while 1 : 
    # 데이터 저장 (dataToCsv)
    f = open('keyword_url2.csv', 'a', encoding='utf-8-sig', newline='')
    writer = csv.writer(f)
    
    with open('keyword_list_sh.pkl', 'rb') as f2:
        keyword_list_sohee = pickle.load(f2)
        
    keyword = keyword_list_sohee[0]
    
    url = 'https://search.naver.com/search.naver?where=blog&query=부산여행 ' + keyword + '&sm=tab_opt&nso=so:r,p:from20220101to20221031'

    driver.get(url)
    time.sleep(3)

    n = 2
    i = 0
    while i < n:
        driver.execute_script("window.scrollTo(0, 99999999)")
        time.sleep(1)
        i = i+1
        
    # 블로그 글 url들 수집
    url_list = []
    title_list = []

    # URL_raw 크롤링 시작
    try :
        article_raw = driver.find_elements(By.CSS_SELECTOR, ".api_txt_lines.total_tit")
        article_raw[0]

        # 크롤링한 url 정제 시작
        for article in article_raw:
            url = article.get_attribute('href')   
            writer.writerow([keyword, url])
            cnt += 1
            if cnt == 50 : break

    except : pass
    cnt = 0
    del keyword_list_sohee[0] 
    with open('keyword_list_sh.pkl', 'wb') as f3:
            pickle.dump(keyword_list_sohee, f3)

IndexError: list index out of range

In [36]:
start_iloc = 0
with open('start_iloc_sohee.pkl', 'wb') as f3:
    pickle.dump(start_iloc, f3)

In [37]:
# blog_content_crawler

with open('start_iloc_sohee.pkl', 'rb') as f2:
    start_iloc = pickle.load(f2)

f = open('blog_content_sohee.csv', 'a', encoding='utf-8-sig', newline='')
writer = csv.writer(f)

df = pd.read_csv('keyword_url2.csv', names=['keyword', 'url'])
print(df)

for i in range(start_iloc, len(df['keyword'])):
    keyword = df.iloc[i,0]
    url = df.iloc[i,1]
    driver.get(url)   # 글 띄우기
    
    try: 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        # 제목 크롤링
        overlays = ".se-module.se-module-text.se-title-text"                  
        title = driver.find_element(By.CSS_SELECTOR, overlays).text
        
        # 날짜 크롤링
        overlays = ".se_publishDate.pcol2"    
        datetime = driver.find_element(By.CSS_SELECTOR, overlays).text

        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"
        # .se-module.se-module-text                                 
        contents = driver.find_elements(By.CSS_SELECTOR, overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)

        content_str = ' '.join(content_list)
        
        # 태그 크롤링
        overlays = ".item.pcol2.itemTagfont._setTop"
        tags = driver.find_elements(By.CSS_SELECTOR, overlays)

        tag_list = []
        for tag in tags:
            tag_list.append(tag.text)
        
        tag_str = ' '.join(tag_list)
        
        # print(df.iloc[i])
        
        writer.writerow([keyword, url, title, datetime, content_str, tag_str])
        
        title, datetime, content_str, tag_str = '', '', '', ''
        
        time.sleep(1)
        
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        print(str(i)+' failed')  
        writer.writerow([keyword, url, '', '', '', ''])
        time.sleep(1)
        continue
    
    with open('start_iloc_sohee.pkl', 'wb') as f3:
            pickle.dump(i, f3)
    print(i)
    
f.close()

    keyword                                                url
0       금정산          https://blog.naver.com/agnea/222713544710
1       금정산       https://blog.naver.com/0kvelys2/222909466613
2       금정산  https://blog.naver.com/outdoor_jenny/222772691361
3       금정산    https://blog.naver.com/busanto1115/222817353704
4       금정산          https://blog.naver.com/koega/222754660067
..      ...                                                ...
643   학리방파제     https://blog.naver.com/hyemin9133/222686310615
644   학리방파제       https://blog.naver.com/sbne1001/222800030131
645   학리방파제   https://blog.naver.com/hihaetae1106/222833850087
646   학리방파제         https://blog.naver.com/qufl59/222664193569
647   학리방파제        https://blog.naver.com/i_nyong/222864413464

[648 rows x 2 columns]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41 failed
42
43
44
45
46
47
48
49
50 failed
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


In [38]:
import pandas as pd

In [42]:
df4=pd.read_csv('./blog38_62.csv',encoding='utf-8',index_col=0)
df4

,keyword,content
38,대천천,부산에 사는 친구을 만나서 점심을 먹고 근처에 있는 대천천 누리길로 산책을 갔어요...
39,대항 어촌체험 휴양마을,안녕하세요 부산어촌특화지원센터 서포터즈 5기 고현수입니다 부산에는 여러 개의 섬이 ...
40,대항선착장,부산 가덕도 대항전망대 외양포 지난 추석에 시부모님과 경치좋은 가덕도에서 드라이브겸...
41,더베이101,101 볼일이 있어서 급 부산 1박2일 내려간 김에 바람도 쐬고 맛난 것도 먹고 ...
42,델타루 (에코델타시티 전망대),작년 겨울이었던 2021년 12월 한창 에코델타시티 단독주택용지 일반분양에 관심이 ...
43,동래온천 노천족탕,부산 동래온천 우체국 동래온천 노천족탕 날짜 2022 03 29 스탬프위치 우편 취...
44,동래온천길(온천천 산책코스),온천천 시민공원을 따라 스탬프 투어가 생겼더라고요 지나가면서 흘려보다가 이번에 친구...
45,동래향교,어버이날을 맞이해 내려온 부산 미리 예약해둔 한정식 식당에서 식사를 하기로 했다 ...
46,동백공원,아침 일찍 일어나 숙소 주변에서 산책하기 좋은 곳 부산 여행 부산 해운대 동백섬 동...
47,마하사(부산),부산 여행 중 택사기사님이 추천해 주신 곳이이에요 부산 황령산은 예전에 스키장이 ...


In [46]:
df4.loc[[51]].to_csv('./blog51.csv',encoding='utf-8',index=False)